# Only the updated data, only June.

In [ ]:
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

In [ ]:
src = '/media/s/Linux_storage/Analyse_Verkehrswende_Transformation/Data/Updates'
hashtags = ['WissZeitVG', 'FristIstFrust', '95vsWissZeitVG',
            'ACertainDegreeOfFlexibility', 'IchBinHanna']

In [ ]:
def get_tweet_frequency(src, filename, date_cutoff=pd.to_datetime('2018-01-01')):
    data = pd.read_csv(join(src, '{}.csv'.format(filename)), low_memory=False)
    print('{}: {} Tweets'.format(filename, len(data)))
    data['created_at'] = pd.to_datetime(data['created_at'])
    data['created_at_date'] = data['created_at'].dt.date
    data = data[data['created_at_date'] >= date_cutoff]
    
    frequency = data[['created_at_date', 'id']]\
        .groupby('created_at_date')\
        .agg('count')\
        .rename(columns={'id':'tweet_count'})
    
    return frequency

In [ ]:
colors = ['grey', '#ffc996', '#ff8474', '#9f5f80', '#583d72' ]

fig, ax = plt.subplots()
for ht, color in zip(hashtags, colors):
    frequency = get_tweet_frequency(src, ht)
    frequency_avg = frequency['tweet_count'].rolling(1).mean().dropna()
    ax.plot(frequency_avg.index, frequency_avg.values, color=color,
            label='#{}'.format(ht), linewidth=2)
    
ax.set_yscale('log')
ax.set_xlim(pd.to_datetime('2021-06-08'), pd.to_datetime('2021-06-21'))
ax.set_ylim(ymax=1e6)
ax.set_xticks([pd.to_datetime(t) for t in ['2021-06-01', '2021-06-10', '2021-06-21',]])
ax.tick_params(axis='x', rotation=45)
ax.set_ylabel('log(Anzahl Tweets / Tag)', fontsize=16)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.legend()

fig.tight_layout()

plt.savefig('../tweet_frequency_short.png', dpi=300)